# Challenge Data Engineer - LATAM - 2024

Postulante: Diego Zimmerman

Email: dzimmerman2611@gmail.com

Empresa: Option

# Objetivo del challenge

    Se me proporcionó un archivo JSON (newline-delimited JSON) llamado "farmers-protest-tweets-2021-2-4". Este archivo contiene aproximadamente 117mil registros. Cada uno de estos registros es un objeto proveniente de la API de Twiter con información acerca del tweet.

    Frente a esto se me pidió responder a tres preguntas:

        1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.
        2. Los top 10 emojis más usados con su respectivo conteo.
        3. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.
    Para cada una de estas consignas se pidió que se implementaran dos soluciones, una enfocada en la optimización del tiempo de ejecución y la otra enfocada en la utilización de memoria.

## Comentarios adicionales

    Además del archivo JSONL se entregó una pequeña estructura de proyecto con algunos archivos. Entre esos archivos (los cuales se encuentran en la carpeta `src/` del proyecto) ya estaban pre-definidas las funciones que se debían completar para responder cada una de las preguntas anteriores (en sus dos versiones).




# Desarrollo del Challenge

A continuación voy a explicar cada uno de los ítems y la forma en que lo decidí resolver. Si bien esta Notebook servirá para entender el enfoque tomado, el código se encuentra documentado y las mismas justificaciones que daré a continuación se encuentran en cada una de las funciones y/o modulos implementados. Podrán existir detalles que no estén en ambos lados pero los ítems principales si lo están.

Lo primero que hice fue revisar el archivo JSON para poder entender cómo estaba compuesto el mismo. Aquí presté atención a los pares clave-valor de cada JSON. 

Luego ingresé a la documentación oficial de la API de Twitter para contrastar lo que estaba observando con lo que decía la documentación.
Esto lo hice para poder asegurar que la estructura de los JSONs serían constantes y poder evaluar mejor qué método de lectura sería mejor para este tipo de archivo.

Luego comencé a pensar en cada función en particular

In [7]:
## Imports y variables comunes

In [23]:
from app.constants import DATA_DIR
from src.q1_memory import q1_memory
from src.q1_time import q1_time
from src.q2_memory import q2_memory
from src.q2_time import q2_time
from src.q3_memory import q3_memory
from src.q3_time import q3_time
from app.main import Logger

Logger()

In [10]:
FILE_PATH = DATA_DIR / "farmers-protest-tweets-2021-2-4.json"

In [12]:
%load_ext memory_profiler

## Top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.

### **Enfoque en eficiencia de memoria**

Con el objetivo de eficientizar el uso de memoria, esta función itera el archivo JSONL línea por línea. Procesar línea por línea implica procesar un JSON a la vez.

El objetivo de esto es evitar cargar en memoria la totalidad del archivo y en cambio procesar cada JSON por separado. Una vez que se terminó de procesar un JSON, guardar lo que sea necesario en memoria y descartar lo demás. Recién ahí continuar con la siguiente línea del archivo (es decir, el siguiente JSON).

**Explicación de función**

En esta función decidí hacer una doble lectura del archivo con la intención de optimizar aún más el uso de la memoria. Para ello describiré los dos enfoques posibles:
1. Se puede leer una única ver el archivo (leyendo línea por línea) y que a la misma vez que se está contando la cantidad de tweets por día se esté contando la cantidad de tweets que cada usuario hizo para los distintos días.
2. Se puede leer una primera vez el archivo (leyendo línea por línea) y contar la cantidad de tweets para cada fecha. Una vez finalizada esta cuenta, encontrar los 10 días con más tweets. Una vez finalizada esta búsqueda, leeré por segunda vez el archivo (leyendo línea por línea) a fin de contar la cantidad de tweets de cada usuario. La diferencia es que ahora solo contaré los tweets de usuarios que solo hayan publicado tweets en las fechas que salieron dentro del TOP 10.

Como se puede observar, el segundo enfoque permite que guarde en memoria la cuenta de tweets de usuarios que efectivamente publicaron algo en los días que salieron en el TOP 10 y no tener que almacenar la cuenta para todos los demás días que no son de interés en esta respuesta.

**NOTAS SOBRE EL CODIGO**
1. Se evitaron variables intermedias y se priorizó anidar calculos a fin de evitar utilizar espacios de memorias para almacenar estas variables intermedias.
2. Se decidió utilizar el método `nlargest` de la líbrería `heapq` frente a otros enfoques como `sorted()` dado que este método evita tener que cargar el listado completo de elementos en memoría para recién poder ordenarlos. El método utilizado permite ir guardando en memoria solo una cantidad N de elementos (en este caso 10) mientras se va iterando sobre el resto del listado (cargando un elemento a la vez en memoria).
3. Se hace uso de `del` para eliminar variables intermedias que eran necesarias pero que después dejan de serlo (ya que su propósito fue cumplido) 

Veamos el output de la función

In [24]:
q1_memory(FILE_PATH)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Ahora le haremos el perfilado de tiempo y de consumo de memoria para después poder comparar con el otro enfoque

#### **Perfilado de tiempo**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

#### **Perfilado de memoria**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

### **Enfoque en eficiencia de tiempo**

Con el objetivo de eficientizar el tiempo, esta función busca trabajar con todo el archivo JSONL de una sola vez, evitando tener que procesar el mismo línea por línea. Si bien esto tiene un efecto significativo en memoria, es importante destacar que aquí asumí que este código siempre se ejecutará en una PC con capacidad de cómputo y memoria suficiente.

**Explicación de función**

Si bien la lectura línea por línea también fue necesaria aquí (dada la estructura JSONL), en esta oportunidad decidí cargar en memoria todas las líneas del archivo antes de empezar a procesar los datos.

De esta manera, cargué todos los JSONs dentro del archivo en una lista que luego convertí en un DataFrame de `polars`. Una vez hecho esto realicé operaciones sencillas de agrupación (group by) y agregación (count) para llegar a la respuesta deseada.
**NOTAS SOBRE EL CODIGO**1. a.
Utilizo la librería orjson por lo que se detalla en la sección "Observaciones generales del Challenge" más abajo en esta notebook.


Veamos el output de la función

In [21]:
q1_time(FILE_PATH)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

#### **Perfilado de tiempo**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

#### **Perfilado de memoria**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

## Los top 10 emojis más usados con su respectivo conteo.

### **Enfoque en eficiencia de memoria**

Con el objetivo de eficientizar el uso de memoria, esta función itera el archivo JSONL línea por línea. Procesar línea por línea implica procesar un JSON a la vez.

El objetivo de esto es evitar cargar en memoria la totalidad del archivo y en cambio procesar cada JSON por separado. Una vez que se terminó de procesar un JSON, guardar lo que sea necesario en memoria y descartar lo demás. Recién ahí continuar con la siguiente línea del archivo (es decir, el siguiente JSON).

**Explicación de función**

Para poder encontrar los emojis decidí hacer uso de expresiones regulares (RegEx). Para esto definí la constante `EMOJI_PATTERN` dentro del módulo `app.constants` que sirve para encontrar los emojis dentro de esos valores pre-definidos.

Al cargar línea por línea el archivo, hice uso de esta constante que ya tiene definido el pattern para la expresión regular e hice uso del método `findall()`. Dado que los tweets son textos medianamente corto, el utilizar `findall()` resulta una estrategia acertada. Si el texto fuese más largo, debería evaluarse el uso de `finditer()` el cual puede ayudar a alivianar la carga en memoria en caso de que se encontrasen muchos emojis en un único tweet.

**NOTAS SOBRE EL CODIGO**
1. Se evitaron variables intermedias y se priorizó anidar calculos a fin de evitar utilizar espacios de memorias para almacenar estas variables intermedias.
2. Se decidió utilizar el método `nlargest` de la líbrería `heapq` frente a otros enfoques como `sorted()` dado que este método evita tener que cargar el listado completo de elementos en memoría para recién poder ordenarlos. El método utilizado permite ir guardando en memoria solo una cantidad N de elementos (en este caso 10) mientras se va iterando sobre el resto del listado (cargando un elemento a la vez en memoria).

Veamos el output de la función

In [25]:
q2_memory(FILE_PATH)

[('🙏', 1940),
 ('❤', 1397),
 ('🌾', 523),
 ('💚', 492),
 ('😂', 488),
 ('👍', 458),
 ('👉', 450),
 ('✊', 425),
 ('🇮🇳', 407),
 ('🙏🙏', 393)]

#### **Perfilado de tiempo**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

#### **Perfilado de memoria**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

### **Enfoque en eficiencia de tiempo**

Con el objetivo de eficientizar el tiempo, esta función busca trabajar con todo el archivo JSONL de una sola vez, evitando tener que procesar el mismo línea por línea. Si bien esto tiene un efecto significativo en memoria, es importante destacar que aquí asumí que este código siempre se ejecutará en una PC con capacidad de cómputo y memoria suficiente.

**Explicación de función**

Si bien la lectura línea por línea también fue necesaria aquí (dada la estructura JSONL), en esta oportunidad decidí cargar en memoria todas las líneas del archivo antes de empezar a procesar los datos.

Como el uso de memoria no es una limitación en este caso, lo que haremos será cargar todos los tweets en un solo `string` y luego procesar este string con una Expresión Regular (RegEx) que contendrá todos los emojis que se desean encontrar.

Luego de obtener la lista de emojis totales, utilizaremos un Contador para devolver los 10 primeros de estos y su correspondiente número de apariciones.

**NOTAS SOBRE EL CODIGO**
1. En este caso decidi usar la clase `Counter` la cual resuelve de manera eficiente y sencilla las operaciones de cuenta.
2. Utilizo la librería `orjson` por lo que se detalla en la sección "Observaciones generales del Challenge" más abajo en esta notebook.


Veamos el output de la función

In [26]:
q2_time(FILE_PATH)

[('🙏', 1940),
 ('❤', 1397),
 ('🌾', 523),
 ('💚', 492),
 ('😂', 488),
 ('👍', 458),
 ('👉', 450),
 ('✊', 425),
 ('🇮🇳', 407),
 ('🙏🙏', 393)]

##### **Perfilado de tiempo**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

#### **Perfilado de memoria**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

## El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.

### **Enfoque en eficiencia de memoria**

Con el objetivo de eficientizar el uso de memoria, esta función itera el archivo JSONL línea por línea. Procesar línea por línea implica procesar un JSON a la vez.

El objetivo de esto es evitar cargar en memoria la totalidad del archivo y en cambio procesar cada JSON por separado. Una vez que se terminó de procesar un JSON, guardar lo que sea necesario en memoria y descartar lo demás. Recién ahí continuar con la siguiente línea del archivo (es decir, el siguiente JSON).

**Explicación de función**

Para poder encontrar los emojis decidí hacer uso de expresiones regulares (RegEx). Para esto definí la constante `EMOJI_PATTERN` dentro del módulo `app.constants` que sirve para encontrar los emojis dentro de esos valores pre-definidos.

Al cargar línea por línea el archivo, hice uso de esta constante que ya tiene definido el pattern para la expresión regular e hice uso del método `findall()`. Dado que los tweets son textos medianamente corto, el utilizar `findall()` resulta una estrategia acertada. Si el texto fuese más largo, debería evaluarse el uso de `finditer()` el cual puede ayudar a alivianar la carga en memoria en caso de que se encontrasen muchos emojis en un único tweet.

**NOTAS SOBRE EL CODIGO**
1. Se evitaron variables intermedias y se priorizó anidar calculos a fin de evitar utilizar espacios de memorias para almacenar estas variables intermedias.
2. Se decidió utilizar el método `nlargest` de la líbrería `heapq` frente a otros enfoques como `sorted()` dado que este método evita tener que cargar el listado completo de elementos en memoría para recién poder ordenarlos. El método utilizado permite ir guardando en memoria solo una cantidad N de elementos (en este caso 10) mientras se va iterando sobre el resto del listado (cargando un elemento a la vez en memoria).

Veamos el output de la función

In [27]:
q3_memory(FILE_PATH)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

#### **Perfilado de tiempo**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)s

#### **Perfilado de memoria**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

### **Enfoque en eficiencia de tiempo**

Con el objetivo de eficientizar el tiempo, esta función busca trabajar con todo el archivo JSONL de una sola vez, evitando tener que procesar el mismo línea por línea. Si bien esto tiene un efecto significativo en memoria, es importante destacar que aquí asumí que este código siempre se ejecutará en una PC con capacidad de cómputo y memoria suficiente.

**Explicación de función**

Si bien la lectura línea por línea también fue necesaria aquí (dada la estructura JSONL), en esta oportunidad decidí cargar en memoria todas las líneas del archivo antes de empezar a procesar los datos.

Lo que haremos es cargar cada JSON (cada línea del JSONL) y extraer de él el campo `mentionedUsers`. Este campo es a su vez una lista de diccionarios donde cada diccionario son los datos del usuario que fue mencionado en el tweet. De esta manera, al momento de la carga de JSON en memoria, extraemos de este campo `mentionedUsers` todos los `usernames` que fueron mencionados en el tweet que estamos procesando.

Una vez procesada la línea en cuestión se actualiza el objeto `Counter` y se sigue procesando la siguiente línea del archivo. De esta manera, cuando se termine de cargar el archivo se tendrá un objeto `Counter` con la cuenta que estamos buscando.

Luego de obtener la lista de emojis totales, utilizaremos un Contador para devolver los 10 primeros de estos y su correspondiente número de apariciones.

**NOTAS SOBRE EL CODIGO**
1. En este caso decidi usar la clase `Counter` la cual resuelve de manera eficiente y sencilla las operaciones de cuenta.
2. Utilizo la librería `orjson` por lo que se detalla en la sección "Observaciones generales del Challenge" más abajo en esta notebook.

**NOTA SOBRE RESULTADO**
Se puede observar que el enfoque para la eficiencia en memoria y en tiempo son muy parecidos, por lo cual no se observan diferencia notorias entre las dos funciones. Ambos resultados son buenos en memoria como en tiempo

Veamos el output de la función

In [28]:
q3_time(FILE_PATH)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

#### **Perfilado de tiempo**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

#### **Perfilado de memoria**

Se deja el detalle capturado por los logs de la App (ejecutado desde conteneder `app_latam`)

# Observaciones generales del Challenge

## CONSIDERACIONES INICIALES

Si bien al momento de eficientizar el tiempo de ejecución de una función una de las primeras cosas que vienen a la cabeza es el uso de Thread o parallelismo, en ese caso se consideró que la aplicación no debía estar pensada para trabajar con archivo mayores al que hoy se tiene (aprox. 400MB). Dada esta condición, pensar en usar Spark o Dask como frameworks de paralelismo y threading no resulta una buena idea ya que muchas veces puede terminar significando un cuello de botella. Al trabajar con archivos chicos como el de este challenge, el querer paralelizar puede terminar resultando en una aplicación más lenta. 

Habiendo asumido lo anterior, se decidió hacer una comparación de tiempos para dos etapas de la función:
1. Etapa de Lectura
2. Etapa de procesamiento

### Etapa de lectura

Se comparó el método `loads()` de la librería `json` y el método `loads()` de la librería `orjson`. Esta última librería resultaba desconocida para mi pero investigando dí con ella y me resultó interesante poder compararlas. Al hacer la comparación vi que `orjson` implicaba un reducción del 37% en la lectura del archivo. Así que decidí ir por esta.

Esto se puede ver en la notebook: `notebooks\JSON read comparison.ipynb`

### Etapa de procesamiento (funciones optimizadas en tiempo)

Se comparó un enfoque implementado en `polars` con otro enfoque implementado en `pandas`. Si bien los tiempos eran muy similares, me resultó interesante plantear una solución con `polars`. Esta librería está tomando mucha importancia en el último tiempo ya que sus métodos están implementados de una manera más eficiente dada su implementación en `Rust`. Si bien `pandas` aún sigue siendo la primera elección y la librería con la comunidad más grande, siempre es interesante evaluar alternativas. Hasta el momento no tuve la oportunidad de trabajar con `polars` en un proyecto real (con `pandas` lo hice en numerosas oportunidades) por lo que me resultó interesante poder implementar algo con esta librería.

Esto se puede ver en la notebook: `notebooks\Q1 processing time.ipynb`

## Tratamiento de Emojis

Si bien tomé conocimiento de la librería `emoji` decidí dejar implementadas las funciones `q2_memory` y `q2_time` utilizando la RegEx. Esto se debe a que hice pruebas con la librería `emoji` (primera vez que la utilizaba)  y me encontré con métodos que trabajaban muy lento y a su vez métodos distintos que debería devolver resultados equivalentes pero no era así. Depende del camino utilizado podía llegar a tener una cuenta distinta de emojis. 

Esto se puede ver en la notebook: `notebooks\Q2 memory efficiency.ipynb`

## Futuros pasos

En caso de que se quiera hacer escalar esta solución se podría pensar en llevar la misma a un entorno Cloud como GCP, AWS o Azure.

Una solución posible para hacer Cloud esta solución podría ser la siguiente:

(Se utilizará **GCP** como ejemplo de referencia)

1. Desplegar una imagen Docker de nuestra solución en el servicio `Cloud Run`. De esta manera por medio de llamadas HTTP podríamos ejecutar las distintas funciones que tenemos. De esta manera aprovecharíamos el servicio serverless, pudiendo escalar vertical y horizontalmente de acuerdo a la configuración que nosotros asignemos y a lo que exija el sistema al querer procesar el archivo.

Esto podría tener un impacto positivo en la optimización del tiempo de ejecución.

**Nota**: Para hacer eso previamente deberemos implementar una pequeña API capaz de entender la requests.

2. Procesar el archivo JSON previo a la ejecución de la funciones, parseándolo adecuadamente y almacenándolo en servicios Cloud que permita acceder de manera rápida y eficiente a los datos (como a su vez procesarlos). Algunas opciones posible son BigQuery y BigQuery (si es que el volumen de datos crece significativamente). En el caso de BigQuery, podríamos parsear el JSON para dejarlo de forma tabular con los datos que nos interesan y luego con consultas SQL eficientes calcular las distintas cosas que necesitamos.

**Nota**: Aquí incurrimos en un costo de almacenamiento y procesamiento extra (si el volumen de datos es bajo se entre en la Free Tier).

3. Podemos evaluar el uso de Dataflow para procesar el archivo
4. Podemos evaluar armar un pipeline de transformación con etapas de staging que permita crear checkpoints para el procesamiento de los datos, evitando así tener que reprocesar archivos enteros. Para esto podemos hacer uso de BigQuery pero principalmente pensaría en Cloud Storage en primera instancia ya que nuestro input está en formato JSON.
5. En caso de querer armar una App con mucha demanda podemos hacer uso del servicio Pub/Sub y así poder enviar solicitudes de procesamiento a nuestra solución indicando, por ejemplo, de dónde levantar el archvio y qué procesamiento aplicar.

Con esto lo que quiero destacar es que la solucion aquí presentada puede crecer en gran medida. Este crecimiento vendrá establecido por la necesidad del negocio, los objetivos del proyecto y el presupuesto del mismo.